In [ ]:
from v2realbot.tools.loadbatch import load_batch
from v2realbot.utils.utils import zoneNY, DATA_DIR
import pandas as pd
import numpy as np
import vectorbtpro as vbt

vbt.settings.set_theme("dark")
vbt.settings['plotting']['layout']['width'] = 1280
vbt.settings.plotting.auto_rangebreaks = True
# Set the option to display with pagination
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_rows', 10)  # Number of rows per page

In [ ]:
batch_id = "e44a5075"
# res, df = load_batch(batch_id=batch_id,
#                      space_resolution_evenly=False,
#                      indicators_columns=["Rsi14"],
#                      main_session_only=True)
# if res < 0:
#     print("Error" + str(res) + str(df))
# df = df["bars"]
# df.info()
# df.head()
# #df.ptable()
# df.to_pickle(DATA_DIR+"/"+f'{batch_id}.pickle')

## FILTERING

In [ ]:
df = pd.read_pickle(DATA_DIR+"/"+f'{batch_id}.pickle')
df

In [ ]:

#naloadujeme do vbt symbol as column
basic_data = vbt.Data.from_data({"BAC": df}, tz_convert=zoneNY)
start_date = pd.Timestamp('2024-03-12 09:30', tz=zoneNY)
end_date = pd.Timestamp('2024-03-13 16:00', tz=zoneNY)

#filter date
#basic_data = basic_data.transform(lambda df: df[df.index.date == start_date.date()])
#filter range
basic_data = basic_data.transform(lambda df: df[(df.index >= start_date) & (df.index <= end_date)])
#filtered_data = basic_data.transform(lambda df: df[(df.index >= start_date) & (df.index <= end_date)])
# #range filtered_data = data[(data.index >= start_date) & (data.index <= end_date)
#df.between_time('09:30', '16:00')
#(df.index.time >= pd.Timestamp('09:30').time()) & (df.index.time <= pd.Timestamp('16:00').time())

In [ ]:
# basic_data.data["BAC"]
rsi14

In [ ]:
#b = filtered_data.get().iloc[100:200] #b[["Open","High"]]
rsi14 = basic_data.data["BAC"]["Rsi14"].rename("Rsi14")
#create subploit
fig = vbt.make_subplots(rows=2, cols=1, shared_xaxes=True, specs=[[{"secondary_y": True}], [{"secondary_y": False}]])
rsi14.vbt.plot(add_trace_kwargs=dict(row=1, col=1, secondary_y=True),fig=fig)
basic_data.data["BAC"].vbt.ohlcv.plot(add_trace_kwargs=dict(row=1, col=1, secondary_y=False), fig=fig)


In [ ]:
run_rsi = vbt.talib_func("rsi")
rsi_new = run_rsi(basic_data.vwap, timeperiod=15)
rsi_new = rsi_new.fillna(0)
# print(rsi_new)
# print(dir(rsi_new))
rsi14 = basic_data.data["BAC"]["Rsi14"]
# print(rsi14)

#zkombinujeme do stejneho dataframe skrz sloupce (axis1)
# combined_df = pd.concat([rsi_new, rsi14], axis=1)
# combined_df

#create subplot
fig = vbt.make_subplots(rows=1, cols=1, shared_xaxes=True)

plot_rsi = vbt.talib_plot_func("rsi")
plot_rsi(rsi_new, fig=fig)
plot_rsi(rsi14, fig=fig)
fig.show()

In [ ]:
entries = rsi_new.vbt.crossed_below(30)
entries

In [ ]:
exits = rsi_new.vbt.crossed_above(70)
exits  

In [ ]:
def plot_rsi(close, rsi, entries, exits):
    fig = vbt.make_subplots(rows=2, cols=1, shared_xaxes=True, specs=[[{"secondary_y": True}], [{"secondary_y": False}]])
    basic_data.data["BAC"].vbt.ohlcv.plot(add_trace_kwargs=dict(row=1, col=1, secondary_y=True),fig=fig)
    rsi.vbt.plot(fig=fig, add_trace_kwargs=dict(row=1, col=1, secondary_y=False), trace_kwargs=dict(line=dict(color='grey', width=1.5)))
    #close.vbt.plot(fig=fig, add_trace_kwargs=dict(row=1, col=1, secondary_y=True))
    entries.vbt.signals.plot_as_entries(y=close, fig=fig, add_trace_kwargs=dict(row=1, col=1, secondary_y=True))
    exits.vbt.signals.plot_as_exits(y=close, fig=fig, add_trace_kwargs=dict(row=1, col=1, secondary_y=True))
    return fig

close = basic_data.get("Close")

print(entries)

plot_rsi(close, rsi_new, entries, exits).show()

clean_entries, clean_exits = entries.vbt.signals.clean(exits) 

plot_rsi(close, rsi_new, clean_entries, clean_exits).show()


In [ ]:
clean_entries.vbt.signals.total() 
clean_exits.vbt.signals.total() 

In [ ]:
index = basic_data.wrapper.index

#minutes from market open
market_open_time = pd.to_timedelta('09:30:00')

# Calculate the market open datetime for each day
market_opens = index.normalize() + market_open_time

minutes_from_open = (index - market_opens).total_seconds() / 60
# Ensuring the result is a Series
minutes_from_open = pd.Series(minutes_from_open, index=index)

#minutes_from_open.values


In [ ]:
symbol_wrapper = basic_data.get_symbol_wrapper()

@vbt.njit
def elapsed_minutes_from_open_nb(time_in_ns):
    market_opens_in_minute = 570  # 9 hours * 60 minutes + 30 minutes
    current_minute = vbt.dt_nb.hour_nb(time_in_ns) * 60 + vbt.dt_nb.minute_nb(time_in_ns)
    #print("current_minutes", current_minutes)
    # Calculate elapsed minutes since market open at 9:30 AM
    minutes_from_open = current_minute - market_opens_in_minute
    print( "elapsed_from_open", minutes_from_open)
    return minutes_from_open if minutes_from_open >= 0 else 0

@vbt.njit
def entry_place_func_nb(c, low, close, time_in_ns, rsi14, window_open, window_close):
    # if c.from_i == 0:  # (1)!
    #     c.out[0] = True
    #     return -1
    # print("ted")
    # print(c.from_i)
    #exit_i = c.from_i - c.wait  # (2)!
    #exit_price = close[exit_i, c.col]  # (3)!
    #hit_price = exit_price * (1 - th)
    market_open_minutes = 570  # 9 hours * 60 minutes + 30 minutes

    for out_i in range(len(c.out)):
        i = c.from_i + out_i

        current_minutes = vbt.dt_nb.hour_nb(time_in_ns[i]) * 60 + vbt.dt_nb.minute_nb(time_in_ns[i])
        #print("current_minutes", current_minutes)
        # Calculate elapsed minutes since market open at 9:30 AM
        elapsed_from_open = current_minutes - market_open_minutes
        elapsed_from_open = elapsed_from_open if elapsed_from_open >= 0 else 0
        #print( "elapsed_from_open", elapsed_from_open)

        #elapsed_from_open = elapsed_minutes_from_open_nb(time_in_ns) 
        in_window = elapsed_from_open > window_open and elapsed_from_open < window_close
        #print("in_window", in_window)
        # if in_window:
        #     print("in window")

        if in_window and rsi14[i] > 60: # and low[i, c.col] <= hit_price: # and hour == 9:  # (4)!
            return out_i
    return -1

#whether the date changed
# day_changed_nb(
#     ts1,
#     ts2
# )

# h_ns(ts1) int


@vbt.njit
def exit_place_func_nb(c, high, close, time_index, tp, sl):  # (5)!
    entry_i = c.from_i - c.wait
    entry_price = close[entry_i, c.col]
    hit_price = entry_price * (1 + tp)
    stop_price = entry_price * (1 - sl)
    for out_i in range(len(c.out)):
        i = c.from_i + out_i

        


        last_bar_of_day = vbt.dt_nb.day_changed_nb(time_index[i], time_index[i + 1])

        #print(next_day)
        if last_bar_of_day: #pokud je dalsi next day, tak zavirame posledni
            print("ted",out_i)
            return out_i
        if high[i, c.col] >= hit_price or close[i, c.col] <= stop_price :
            return out_i
    return -1

index = basic_data.wrapper.index

#minutes from market open
market_open_time = pd.to_timedelta('09:30:00')

# Calculate the market open datetime for each day
market_opens = index.normalize() + market_open_time

minutes_from_open = (index - market_opens).total_seconds() / 60

print(minutes_from_open)


#index 9:30 az 10:00
time_entry_window = ((index.time >= pd.Timestamp("09:30:00").time())&
                        (index.time <= pd.Timestamp("14:00:00").time()))


print(time_entry_window)
print(rsi_new)
rsi_entries = rsi_new.vbt.crossed_below(64)
rsi_entries = rsi_entries < 40
rsi_entries_array = rsi_entries.vbt.to_1d_array()
print(rsi_entries_array)


entries, exits = vbt.pd_acc.signals.generate_both(  # (6)!
    symbol_wrapper.shape,
    entry_place_func_nb=entry_place_func_nb,
    #timeindex to ns to numba
    entry_place_args=(vbt.Rep("low"), vbt.Rep("close"), vbt.dt.to_ns(basic_data.wrapper.index), vbt.Rep("rsi14"), 0, 380),  # (7)!
    exit_place_func_nb=exit_place_func_nb,
    exit_place_args=(vbt.Rep("high"), vbt.Rep("close"), vbt.dt.to_ns(basic_data.wrapper.index), 0.001, 0.001),
    wrapper=symbol_wrapper,
    broadcast_named_args=dict(  # (8)!
        high=basic_data.get("High"),
        low=basic_data.get("Low"),
        close=basic_data.get("Close"),
        rsi14=basic_data.get("Rsi14"),
        window_open=10,
        window_close=60
    ),
    broadcast_kwargs=dict(post_func=np.asarray)  # (9)!
)


plot_rsi(close, rsi_new, entries, exits).show()

# fig = basic_data.plot(
#     symbol="BAC", 
#     ohlc_trace_kwargs=dict(opacity=0.5), 
#     plot_volume=False
# )

# #rsi_entries.vbt.plot(fig=fig)
# entries.vbt.signals.plot_as_entries(
#     y=close, fig=fig)
# exits.vbt.signals.plot_as_exits(
#     y=close, fig=fig)
# fig.show()  # (10)!